# AICore-Bridge

> Bridge between Stactics AICore framework and Wodan/Conan processor modules

In [ ]:
#| default_exp aicorebridge

In [ ]:
#| hide
from nbdev.showdoc import *
import addroot

In [ ]:
#| hide
addroot.sys.path.append(addroot.os.path.join(addroot.project_root, 'corebridge'))

In [ ]:
#| export

import typing
import logging
import inspect
import datetime
import json
import pandas as pd
import numpy as np

from fastcore.basics import patch_to, patch
from corebridge.core import *


Loading corebridge.core from /home/fenke/repos/corebridge/corebridge/core.py


In [ ]:
#| export
syslog = logging.getLogger(__name__)

In [ ]:
#| export
try:
    print(f"Loading {__name__} from {__file__}")
except:
    pass

In [ ]:
#| export
class AICoreModule(): pass

In [ ]:
#| export
@patch
def __init__(self:AICoreModule, 
             processor:typing.Callable, # data processing function
             save_dir:str, # path where the module can keep files 
             assets_dir:str,
             *args, **kwargs):
    
    self.init_time = datetime.datetime.utcnow()
    self.save_dir   = save_dir
    self.assets_dir = assets_dir
    self._init_processor(processor)

    self.init_args = args
    self.init_kwargs = kwargs



In [ ]:
#| export
@patch
def _init_processor(
        self:AICoreModule, 
        processor:typing.Callable):
    """Initializes processor related variables on self"""
    
    self.processor = processor
    self.processor_signature = inspect.signature(self.processor)
    self.processor_params = dict(self.processor_signature.parameters)
    self.return_param = self.processor_params.pop('return', None)
    self.data_param, *self.call_params = list(self.processor_params.keys())


In [ ]:
#| export
@patch
def call_processor(self:AICoreModule, calldata, **callargs):
    return self.processor(calldata, **callargs)


In [ ]:
#| export
@patch
def infer(self:AICoreModule, data:dict, *_, **kwargs):
    try:

        msg=[
            f"{self.processor.__name__}({self.processor_signature})",
            f"init_args: {self.init_args}, init_kwargs: {self.init_kwargs}",
        ]

        lastSeen = kwargs.pop('lastSeen', False)
        recordformat = kwargs.pop('format', "records").lower()
        reversed = kwargs.pop('reversed', False)
        timezone = kwargs.get('timezone', 'UTC')
        msg.append(f"lastSeen: {lastSeen}, recordformat: {recordformat}, timezone: {timezone}")

        calldata = self.get_call_data(
            data, 
            recordformat=recordformat,
            timezone=timezone,
            reversed=reversed)
        
        msg.append(f"calldata shape: {calldata.shape}")

        callargs = self.get_callargs(**kwargs)

        for arg, val in callargs.items():
            msg.append(f"{arg}: {val}")
            
        result = self.call_processor(calldata, **callargs)
        msg.append(f"result shape: {result.shape}")

        return {
            'msg':msg,
            'data': timeseries_dataframe_to_datadict(
                result if not lastSeen else result[-1:],
                recordformat=recordformat,
                timezone=timezone,
                reversed=reversed)
        }
    except Exception as err:
        return {
            'msg': f"Unexpected {err=}, {type(err)=}",
            'data': []
        }


In [ ]:
#| export
@patch
def get_callargs(self:AICoreModule, **kwargs):
    "Get arguments for the processor call"

    # Remove null / None values
    kwargs = {k:v for k,v in kwargs.items() if v is not None}
    
    metadata = kwargs.pop('metadata', {}) # TODO: historic metadata

    return {
        K:self.processor_signature.parameters[K].annotation(kwargs.get(K,metadata.get(K, self.init_kwargs.get(K, self.processor_signature.parameters[K].default))))
        for K in self.call_params
    }


In [ ]:
#| export
@patch
def get_call_data(
        self:AICoreModule, 
        data:dict, 
        recordformat='records', 
        timezone='UTC', 
        reversed=False):
    
    "Convert data to the processor signature"

    df = set_time_index_zone(timeseries_dataframe_from_datadict(
        data, ['datetimemeasure', 'time'], recordformat), timezone)

    if reversed:
        df = df[::-1]

    if self.processor_params[self.data_param].annotation == pd.DataFrame:
        return df
    elif len(df.columns) > 1:
        df.index = (df.index - datetime.datetime(1970,1,1, tzinfo=datetime.timezone.utc)) / datetime.timedelta(seconds=1)
        return df.to_records(index=True)
    else:
        df.index = (df.index - datetime.datetime(1970,1,1, tzinfo=datetime.timezone.utc)) / datetime.timedelta(seconds=1)
        return df.reset_index().to_numpy()
        

In [ ]:
import os, addroot
import corebridge

In [ ]:
def test_function(data:pd.DataFrame, anumber:float=0):
    v = 2*anumber
    return data

class TestAICoreModule(AICoreModule):
    def __init__(self, save_dir, *args, **kwargs):
        super().__init__(test_function, save_dir, None, *args, **kwargs)


In [ ]:
save_dir = os.path.join(os.getcwd(), 'cache')
test_module = TestAICoreModule(os.path.join(os.getcwd(), 'cache'), 1, 2, num_1=3, num_2=4)

assert test_module.init_args == (1, 2)
assert test_module.init_kwargs['num_1'] == 3
assert test_module.init_kwargs['num_2'] == 4
assert test_module.save_dir == save_dir

In [ ]:
test_data = [
    dict(datetimeMeasure='2020-04-01T00:01:11.123Z', value=1.1),
    dict(datetimeMeasure='2020-04-02T00:20:00Z', value=2.3),
]
result = test_module.infer(test_data, timezone='Europe/Amsterdam', anumber=None)
print("Test Data\n", json.dumps(test_data, indent=2))
print("Result Message\n", json.dumps(result['msg'], indent=2))
print("Result Data\n", json.dumps(result['data'], indent=2))

Test Data
 [
  {
    "datetimeMeasure": "2020-04-01T00:01:11.123Z",
    "value": 1.1
  },
  {
    "datetimeMeasure": "2020-04-02T00:20:00Z",
    "value": 2.3
  }
]
Result Message
 [
  "test_function((data: pandas.core.frame.DataFrame, anumber: float = 0))",
  "init_args: (1, 2), init_kwargs: {'num_1': 3, 'num_2': 4}",
  "lastSeen: False, recordformat: records, timezone: Europe/Amsterdam",
  "calldata shape: (2, 1)",
  "anumber: 0.0",
  "result shape: (2, 1)"
]
Result Data
 [
  {
    "time": "2020-04-01T02:01:11.123000+02:00",
    "value": 1.1
  },
  {
    "time": "2020-04-02T02:20:00+02:00",
    "value": 2.3
  }
]


In [ ]:
#import corebridge.core

In [ ]:
from corebridge.aicorebridge import AICoreModule

class TestAICoreModule(AICoreModule):
    def __init__(self, save_dir, *args, **kwargs):
        super().__init__(test_function, save_dir, None, *args, **kwargs)
        
test_module = TestAICoreModule(os.path.join(os.getcwd(), 'cache'), 1, 2, num_1=3, num_2=4)

assert test_module.init_args == (1, 2)
assert test_module.init_kwargs['num_1'] == 3
assert test_module.init_kwargs['num_2'] == 4
assert test_module.save_dir == save_dir

Loading core from /home/fenke/repos/corebridge/corebridge/core.py
Loading corebridge.aicorebridge from /home/fenke/repos/corebridge/corebridge/aicorebridge.py


In [ ]:
test_data = [
    dict(datetimeMeasure='2020-04-01T00:01:11.123Z', value=1.1),
    dict(datetimeMeasure='2020-04-02T00:20:00Z', value=2.3),
]
result = test_module.infer(test_data, timezone='Europe/Amsterdam', anumber=None)
print("Test Data\n", json.dumps(test_data, indent=2))
print("Result Message\n", json.dumps(result['msg'], indent=2))
print("Result Data\n", json.dumps(result['data'], indent=2))

Test Data
 [
  {
    "datetimeMeasure": "2020-04-01T00:01:11.123Z",
    "value": 1.1
  },
  {
    "datetimeMeasure": "2020-04-02T00:20:00Z",
    "value": 2.3
  }
]
Result Message
 [
  "test_function((data: pandas.core.frame.DataFrame, anumber: float = 0))",
  "init_args: (1, 2), init_kwargs: {'num_1': 3, 'num_2': 4}",
  "lastSeen: False, recordformat: records, timezone: Europe/Amsterdam",
  "calldata shape: (2, 1)",
  "anumber: 0.0",
  "result shape: (2, 1)"
]
Result Data
 [
  {
    "time": "2020-04-01T02:01:11.123000+02:00",
    "value": 1.1
  },
  {
    "time": "2020-04-02T02:20:00+02:00",
    "value": 2.3
  }
]


### References

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()